# Effect of delaying lockdown

This uses an SEIR model to simulate the difference in total cases when lockdown is imposed at different thresholds of cases.

In [140]:
import numpy as np
from scipy.integrate import odeint
from bokeh.palettes import Dark2
from bokeh.plotting import figure, show, curdoc
from bokeh.models import ColumnDataSource, Patch, NumeralTickFormatter, FuncTickFormatter, DatetimeTickFormatter, Span, Label, Legend, LegendItem, Band
from bokeh.themes import Theme
from bokeh.io import output_notebook
from itertools import cycle
output_notebook()
curdoc().theme = Theme('../russ.yaml')

Loading BokehJS ...

## Parameters

We assume the $R_0$ value to be 2.5, which is reduced to 0.7 when lockdown occurs. Serial interval and incubation period are set roughly from observed numbers for COVID-19. The exact numbers don't make a huge difference to the point I'm trying to make here.

We set the initial conditions of the model to assume 1000 people already have been infected, to make the graph look reasonable. The solver seems to be a little bit sensitive to these initial conditions so I had to tweak things a bit to get a reasonable result. 

In [177]:
R_0 = 2.5
R_lockdown = 0.7
serial_interval = 4
incubation = 5.1

# Initial conditions
N = 50e6                # Population size - not really relevant here as the total number of cases is always much lower
E0 = 500                # Initial number of exposed
I0 = 499                # Initial number of infected
R0 = 1                  # Initial number of recovered (note this is different from R_0...)
S0 = N - I0 - R0 - E0   # Initial number of susceptible (all the rest)

## Model

For the purposes of this graph, we don't need an SEIR model - SIR would do fine - but this is the one I had lying around. It doesn't consider "vital dynamics", which is the fancy name for "people being born and dying for reasons other than the disease" and is not particularly relevant in short timeframes.

Here's maths, mostly for my own benefit:

$$\begin{align}
\frac{dS}{dt} & = -\frac{\beta SI}{N} \\
\frac{dE}{dt} & = \frac{\beta SI}{N} - \sigma E \\
\frac{dI}{dt} & = \sigma E - \gamma I \\
\frac{dR}{dt} & = \gamma I \\
N & = S + E + I + R
\end{align}
$$

* $\beta$ is the rate at which *susceptible* becomes *exposed*.
* $\sigma$ is the rate at which *exposed* becomes *infected*.
* $\gamma$ is the rate at which *infected* becomes *removed*.

We can define these parameters in terms of the incubation period, serial interval, and $R_0$ as:

$$\begin{align}
\sigma & = \frac{1}{\text{incubation period}} \\
\gamma & = \frac{1}{\text{serial interval}} \\
\beta & = \gamma R_0
\end{align}
$$

In [178]:
gamma = 1/serial_interval
sigma = 1/incubation

# Model function to pass to the solver
def model(y, t, N, gamma, sigma, switchpoint):
    S, E, I, R = y
    
    # Introduce lockdown if total infected is greater than the threshold
    if (E + I + R) >= switchpoint:
        R_t = R_lockdown
    else:
        R_t = R_0
    
    beta = gamma * R_t
    
    dSdt = -beta * ((S * I) / N)
    dEdt = ((beta * S * I) / N) - sigma * E
    dIdt = sigma * E - gamma * I
    dRdt = gamma * I
    
    return dSdt, dEdt, dIdt, dRdt

y0 = S0, E0, I0, R0  # Vector of initial conditions which we'll feed into the solver

## Simulate and plot

In [183]:
# Simulate 150 days with 6 points per day (this is mostly to make sure that we have enough temporal resolution for our lockdown markers to line up)
days = 150
t = np.linspace(0, days, days * 6)

fig = figure(width=900, title='Effect of delaying lockdown')
colors = cycle(Dark2[5])

# Run a simulation for each lockdown point
for switchpoint in (1e6, 5e5, 2.5e5, 1.25e5):
    # Solve the model
    ret = odeint(model, y0, t, args=(N, gamma, sigma, switchpoint))
    S, E, I, R = ret.T
    infected = (E + I + R)
    
    fig.line(x=t, y=infected, legend_label=f'{int(switchpoint):,} cases', color=next(colors))
    
    switch_time = t[infected <= switchpoint][-1]
    switch_amount = infected[infected <= switchpoint][-1]
    fig.circle(x=switch_time, y=switch_amount, size=4, color='#555555')
    fig.add_layout(Label(x=switch_time, y=switch_amount, text=f'{int(round(switch_time))} days', x_offset=-45, y_offset=-5,
                             text_font_size=f"10px", text_color='#555555', text_font="Noto Sans",
                             background_fill_color="white", background_fill_alpha=0.7))

fig.yaxis.formatter = NumeralTickFormatter(format="0,0")
fig.yaxis.axis_label = "Cumulative cases"
fig.xaxis.axis_label = f"Days since 1000 cases"
fig.legend.location = "top_left"
fig.xgrid.visible = False
fig.legend.title = 'Lockdown imposed at'

info_label = Label(x=0, y=0, x_units='screen', y_units='screen', text_font='Noto Sans', text_font_size="9px", text_color='#666666',
                      text=f'SEIR model with R={R_0} before lockdown and R={R_lockdown} after lockdown. Russ Garrett / @russss')
fig.add_layout(info_label, 'below')
show(fig)